In [1]:
import dask.dataframe as dd
import dask.bag as da
import os
import json
import dask

from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

2024-04-27 17:29:16,481 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle f0533596b228563fd169fd034dca9324 initialized by task ('shuffle-transfer-f0533596b228563fd169fd034dca9324', 99) executed on worker tcp://127.0.0.1:51606
2024-04-27 17:29:16,888 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle f0533596b228563fd169fd034dca9324 deactivated due to stimulus 'task-finished-1714231756.8873951'
2024-04-27 17:34:20,729 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:51582' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('to_pyarrow_string-540cdaeb234ff1add394e877a6dea74d', 44), ('to_pyarrow_string-540cdaeb234ff1add394e877a6dea74d', 60), ('to_pyarrow_string-540cdaeb234ff1add394e877a6dea74d', 21), ('to_pyarrow_string-540cdaeb234ff1add394e877a6dea74d', 75), ('to_pyarrow_string-540cdaeb234ff1add394e877a6dea74d', 94), ('to_pyarrow_string-540cdaeb234ff1add394e877a6dea74d', 23), ('to_pyarrow_string-540cdaeb234ff1

In [2]:
cluster.dashboard_link

'http://127.0.0.1:8787/status'

In [3]:
os.makedirs('data_ex1', exist_ok=True)              # Create data/ directory

b = dask.datasets.make_people(npartitions=100, records_per_partition=10000)                 # Make records of people
b.map(json.dumps).to_textfiles('data_ex1/*.json')   # Encode as JSON, write to disk

['/Users/stefanotavonatti/Teaching/lab5/data_ex1/00.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/01.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/02.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/03.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/04.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/05.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/06.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/07.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/08.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/09.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/10.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/11.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/12.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/13.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/14.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/15.json',
 '/Users/stefanotavonatti/Teaching/lab5/data_ex1/16.json

In [4]:
people_bag = da.read_text('data/*.json').map(json.loads)
people_bag

dask.bag<loads, npartitions=100>

In [5]:
people_df = people_bag.to_dataframe().persist()
people_df

,age,name,occupation,telephone,address,credit-card
npartitions=100,,,,,,
,int64,string,string,string,string,string
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [6]:
people_df.head()

,age,name,occupation,telephone,address,credit-card
0,90,"['Brett', 'Daugherty']",Postwoman,+18158303371,"{'address': '70 Briarcliff Green', 'city': 'Pl...","{'number': '3413 955825 91568', 'expiration-da..."
1,4,"['Carry', 'Kent']",Stage Manager,+18657290679,"{'address': '718 Morris Estate', 'city': 'Hill...","{'number': '3722 617973 78401', 'expiration-da..."
2,103,"['Margart', 'Stanton']",Bus Valeter,+1-641-392-2680,"{'address': '1265 China Basin Motorway', 'city...","{'number': '3488 869513 35929', 'expiration-da..."
3,116,"['Stewart', 'Delgado']",Ambulance Controller,+1-274-996-4825,"{'address': '661 San Marcos Ferry', 'city': 'C...","{'number': '4628 5138 3022 9544', 'expiration-..."
4,8,"['Stacey', 'Ochoa']",General Practitioner,+14084623918,"{'address': '877 Brotherhood Station', 'city':...","{'number': '3708 259650 35597', 'expiration-da..."


2. **Average people age**

In [7]:
people_df['age'].mean().compute()

59.942509

3. **Most common name**

In [21]:
max(people_bag.map(lambda x: x['name'][0]).frequencies().compute(), key=lambda x: x[1])

('Marlin', 697)

4. **Average age for each profession**

In [22]:
people_df.groupby(['occupation']).agg({'age': 'mean'}).compute()

,age
occupation,
Garage Manager,59.738359
Research Consultant,59.538639
Probation Officer,60.726823
Buyer,60.454976
Quality Technician,60.212010
...,...
Analytical Chemist,59.425721
Turf Accountant,58.112269
Typewriter Engineer,58.225248


4. **How many credit card expire in 2025**

In [25]:
people_bag.map(lambda x: x['credit-card']).take(1)[0]

{'number': '3413 955825 91568', 'expiration-date': '12/21'}

In [29]:
card_count = people_bag.filter(lambda x: x['credit-card']['expiration-date'].split("/")[1] == '25').count()
card_count

In [30]:
card_count.visualize()

In [31]:
card_count.compute()

99668

In [32]:
people_bag.count().compute()

1000000